In [2]:

import numpy as np
from numpy import linalg as la
import pdb
import copy
import itertools
import pdb
import matplotlib
# matplotlib.use('TkAgg')
import matplotlib.pyplot as pl
import pickle
import scipy
import cvxpy as cp
import casadi as ca

In [3]:
class MO_FTOCP(object):

    def __init__(self, N, A, B, Q, R, alpha = 0.99):
        # Define variables
        self.N = N # Horizon Length

        # System Dynamics (x_{k+1} = A x_k + Bu_k)
        self.A = A 
        self.B = B 
        self.n = A.shape[1]
        self.d = B.shape[1]
        
        self.Q = Q
        self.R = R
        
        self.alpha = alpha
        
     
        
        self.Q2= [3]#, 10**1]
        
        
        self.ub_x = 4
        self.lb_x =-4
        self.ub_u = 2
        self.lb_u = -2
    
        
        # Initialize Predicted Trajectory
        self.xPred = []
        self.uPred = []
        self.soc = 100
        
        self.Vt1=0.0
        self.Vt2= [0.0]*(len(self.Q2))

    def solve(self, x0, verbose = False, SS = None, Qfun_1 = None, Qfun_2 = None, CVX =None,
              time= 0, Vp_1 = None, Vp_2 = None):
        """This method solves an FTOCP given:
            - x0: initial condition
            - SS: (optional) contains a set of state and the terminal constraint is ConvHull(SS)
            - Qfun: (optional) cost associtated with the state stored in SS. Terminal cost is BarycentrcInterpolation(SS, Qfun)
        """ 
        
   
        
        # Initialize Variables
        x = cp.Variable((self.n, self.N+1))
        u = cp.Variable((self.d, self.N))
        
        constr = [x[:,0] == x0[:]]
        # If SS is given initialize lambdaVar multipliers used to encorce terminal constraint
        if SS is not None:
            if CVX == True:
                lambVar = cp.Variable((SS.shape[1], 1), boolean=False) # Initialize vector of variables
                c1_s    = cp.Variable(1)
                c2_s    = cp.Variable(len(self.Q2))
            else:
                lambVar = cp.Variable((SS.shape[1], 1), boolean=True) # Initialize vector of variables
        # else:
        #     soc = cp.Variable((1, self.N+1))
        #     constr += [soc[:,0]==self.soc] 

        # State Constraints
        
        for i in range(0, self.N):
            constr += [x[:,i+1] == self.A@x[:,i] + self.B@u[:,i],
                        u[:,i] >= self.lb_u,
                        u[:,i] <= self.ub_u,
                        x[:,i] >= self.lb_x,
                        x[:,i] <=  self.ub_x]
            
            # if SS is None:
            #     constr+=[soc[:,i+1] == soc[:,i] - self.Q2[0]*cp.norm2(x[self.n//2:,i]),
            #              soc[:,i+1]>=0]
            
            

        # Terminal Constraint if SS not empty --> enforce the terminal constraint
        if SS is not None:
            constr += [SS @ lambVar[:,0] == x[:,self.N], # Terminal state \in ConvHull(SS)
                        np.ones((1, SS.shape[1])) @ lambVar[:,0] == 1, # Multiplies \lambda sum to 1
                        lambVar >= 0,
                        c1_s >=0,
                        c2_s >=0] # Multiplier are positive definite

        # Cost Function 
        cost_1 = 0
        cost_2 = [0 for _ in range(len(self.Q2))]
        
        for i in range(0, self.N):
            cost_1 += cp.quad_form(x[:, i],  self.Q) + cp.quad_form(u[:,i], self.R)
            for j in range(len(self.Q2)):
                cost_2[j] += self.Q2[j]*cp.norm2(x[self.n//2:,i])
                        
          

        # Terminal cost if SS not empty
        if SS is not None:
            cost_1 += Qfun_1[0,:] @ lambVar[:,0]  # It terminal cost is given by interpolation using \lambda
            for j in range(len(self.Q2)):
                cost_2[j]+= Qfun_2[j][0,:] @ lambVar[:,0]
        else:
            cost_1+=cp.quad_form(x[:, self.N],  self.Q) 
            for j in range(len(self.Q2)):
                cost_2[j]+= self.Q2[j]*cp.norm2(x[self.n//2:,self.N])

        
        alpha = self.alpha
            
        if Vp_1 is not None and Vp_2 is not None:
            
            if time==0:
          
                cost_0= (1-alpha)*cost_1/Vp_1 + 100*cp.sum_squares(c1_s)+100*cp.sum_squares(c2_s) 
                constr+= [cost_1 <=Vp_1+c1_s]
                stage_cost1 = 0
                stage_cost2 = 0
                
                for j in range(len(self.Q2)):
                    cost_0+=alpha*cost_2[j]/Vp_2
                    constr+=[cost_2[j] <=Vp_2[j]+c2_s]
            else:
                cost_0= (1-alpha)*cost_1/Vp_1 + 100*cp.sum_squares(c1_s)+100*cp.sum_squares(c2_s)
                
                stage_cost1 = cp.quad_form(self.xPred[:, 0],  self.Q) + cp.quad_form(self.uPred[:,0], self.R)
                stage_cost2=[self.Q2[j]*cp.norm2(self.xPred[self.n//2:,0]) for j in range(len(self.Q2))]
               
                constr+= [(cost_1+stage_cost1)<=self.Vt_1 + c1_s]
           
                for j in range(len(self.Q2)):
                    cost_0+=alpha*cost_2[j]/Vp_2
                    constr+=[(cost_2[j]+stage_cost2[j]) <=self.Vt_2[j] +c2_s] #trajectory cost Lyap constraint
                   

        else:
            cost_0=cost_1 + cost_2[0]
        
       
            
        # Solve the Finite Time Optimal Control Problem
        problem = cp.Problem(cp.Minimize(cost_0), constr)
  
        # problem.solve(verbose=verbose, solver = cp.ECOS, abstol = 1e-6, abstol_inacc=1e-6) 
        problem.solve(verbose=verbose, solver = cp.ECOS) 
      
        
        # Store the open-loop predicted trajectory
        self.xPred = x.value
        self.uPred = u.value
        self.soc  -= self.Q2[0]*cp.norm2(self.xPred[self.n//2:,0]).value
        
        self.Vt_1=cost_1.value
        self.Vt_2=[cost_2[j].value for j in range(len(self.Q2))]
        
        
        if SS is not None:
            self.lamb  = lambVar.value


    def model(self, x, u):
        # Compute state evolution
        return np.dot(self.A,x) + np.squeeze(np.dot(self.B,u))
    
    
    
    
    
class MO_FTOCP_soc(object):

    def __init__(self, N, A, B, Q, R, alpha = 0.99):
        # Define variables
        self.N = N # Horizon Length

        # System Dynamics (x_{k+1} = A x_k + Bu_k)
        self.A = A 
        self.B = B 
        self.n = A.shape[1]
        self.d = B.shape[1]
        
        self.Q = Q
        self.R = R
        
     
        
        self.Q2= [3]#, 10**1]
        
        
        self.ub_x = 4
        self.lb_x =-4
        self.ub_u = 1
        self.lb_u = -1
        
        self.soc = 100
    
        
        # Initialize Predicted Trajectory
        self.xPred = []
        self.uPred = []
        
        
    def solve(self, x0, verbose = False, CVX =None,
              time= 0):
        """This method solves an FTOCP given:
            - x0: initial condition
        """ 
        
        opti = ca.Opti()
        
        # Initialize Variables
        x = opti.variable(self.n, self.N+1)
        u = opti.variable(self.d, self.N)
        soc = opti.variable(1, self.N)
  
        # State Constraints
        opti.subject_to(x[:,0] == x0[:])
       
        for i in range(0, self.N):
            opti.subject_to(x[:,i+1] == self.A@x[:,i] + self.B@u[:,i])
            opti.subject_to(opti.bounded(self.lb_u, u[:,i], self.ub_u))
            opti.subject_to(opti.bounded(self.lb_x, x[:,i], self.ub_x))
            
        cost_1 = 0
        soc_consumed_total = 0
        for i in range(0, self.N):
            cost_1 += ca.bilin(self.Q, x[:, i]) + ca.bilin(self.R, u[:,i])
            soc_consumed_stage = self.Q2[0]**2*ca.sumsqr(x[self.n//2:,i])
            opti.subject_to(soc_consumed_stage <=soc[:,i]**2)
            opti.subject_to(soc[:,i]>=0)
            soc_consumed_total+=soc[:,i]
           
           
        opti.subject_to(soc_consumed_total<=self.soc)    
                  
        cost_1+=ca.bilin(self.Q, x[:, self.N]) 
                
        cost_0=cost_1
        
            
        # Solve the Finite Time Optimal Control Problem
        opts_setting = {'ipopt.print_level': 0, 'print_time': 0}
        opti.solver("ipopt", opts_setting)
        # opti.solver("osqp")
        problem = opti.solve()
  
        # problem.solve(verbose=verbose, solver = cp.MOSEK, abstol = 1e-6, abstol_inacc=1e-6) 
        
      
        
        # Store the open-loop predicted trajectory
        self.xPred = problem.value(x)
        self.uPred = problem.value(u)
        self.soc  -= self.Q2[0]*ca.norm_2(self.xPred[self.n//2:,0])
                

    def model(self, x, u):
        # Compute state evolution
        return np.dot(self.A,x) + np.squeeze(np.dot(self.B,u))
    
    

In [4]:
class MO_LMPC(object):

    def __init__(self, ftocp, CVX):
        # Initialization
        self.ftocp = ftocp
        self.SS    = []
        self.socSS = []
        self.uSS   = []
        
        self.Q = ftocp.Q
        self.R = ftocp.R
        self.Q2=ftocp.Q2

        self.it    = 0
        self.CVX = CVX
        
        self.Qfun_1  = []
        self.Qfun_2  = [[] for _ in range(len(self.Q2))]
        
        self.cost_by_iter = []

    def addTrajectory(self, x, u):
        # Add the feasible trajectory x and the associated input sequence u to the safe set
        self.SS.append(copy.copy(x))
        self.uSS.append(copy.copy(u))

        # Compute and store the cost associated with the feasible trajectory
        cost = self.computeCost(x, u)
        self.Qfun_1.append(cost[0])
        [self.Qfun_2[j].append(cost[1][j]) for j in range(len(self.Q2))]
        

        # Augment iteration counter and print the cost of the trajectories stored in the safe set
        self.it = self.it + 1
        print("Trajectory added to the Safe Set. Current Iteration: ", self.it)
        print("Trajectory cost: \n", [self.Qfun_1[i][0] for i in range(0, self.it)])
        print("SOC consumed: \n", [[self.Qfun_2[j][i][0] for j in range(len(self.Q2))] for i in range(0, self.it)])
        
        self.cost_by_iter.append([self.Qfun_1[self.it-1][0], [self.Qfun_2[j][self.it-1][0] for j in range(len(self.Q2))]])

    def computeCost(self, x, u):
        # Compute the cost in a DP like strategy: start from the last point x[len(x)-1] and move backwards
        soc = []
        for i in range(0,len(x)):
            idx = len(x)-1 - i
            
            if i == 0:
                
                cost_1 = [x[idx].T@self.Q@x[idx]]
                cost_2= [[] for _ in range(len(self.Q2))]
                for j in range(len(self.Q2)):
                    cost_2[j]  =[self.Q2[j]*np.linalg.norm(x[idx][2:])]
                    
                soc.append(100-self.Q2[0]*np.linalg.norm(x[i][2:]))
            else:
                cost_1.append(x[idx].T@self.Q@x[idx] + u[idx].T@self.R@u[idx] + cost_1[-1])
                soc.append(soc[-1]-self.Q2[0]*np.linalg.norm(x[i][2:]))
                
                for j in range(len(self.Q2)):
                   
                    cost_2[j].append(self.Q2[j]*np.linalg.norm(x[idx][2:])+ cost_2[j][-1])
        
        self.socSS.append(copy.copy(soc))            
                    
                   

        # Finally flip the cost to have correct order
        return [np.flip(cost_1).tolist(), [np.flip(cost_2[j]).tolist() for j in range(len(self.Q2))]]

    def solve(self, xt, time, Vp_1 = None, Vp_2 = None, verbose = False):

        # Build SS and cost matrices used in the ftocp 
        # NOTE: it is possible to use a subset of the stored data to reduce computational complexity while having all guarantees on safety and performance improvement
        SS_vector = np.squeeze(list(itertools.chain.from_iterable(self.SS))).T # From a 3D list to a 2D array
        Qfun_vector_1 = np.expand_dims(np.array(list(itertools.chain.from_iterable(self.Qfun_1))), 0) # From a 2D list to a 1D array
        Qfun_vector_2 = [np.expand_dims(np.array(list(itertools.chain.from_iterable(self.Qfun_2[j]))), 0) for j in range(len(self.Q2))]
        # Solve the FTOCP.
        

        self.ftocp.solve(xt, verbose, SS=SS_vector, Qfun_1=Qfun_vector_1, Qfun_2=Qfun_vector_2, CVX=self.CVX,
                        time=time, Vp_1= Vp_1, Vp_2 = Vp_2)


        # Update predicted trajectory
        self.xPred= self.ftocp.xPred
        self.uPred= self.ftocp.uPred

## Collecting Iteration 0

In [5]:
# Define system dynamics and cost
A = np.array([[1,0, 0.25, 0],[0,1, 0, 0.25], [0,0,1,0], [0,0,0,1]])
B = np.array([[0.5*(0.25)**2, 0],[0,0.5*(0.25)**2], [0.25, 0], [0,0.25]])

Q=np.diag([0.5,0.5, 0.02, 0.02])
R = np.diag([.6, .6])

np.set_printoptions(precision=2)

print("Computing first feasible trajectory")

# Initial Condition
x0 = np.array([3.0, 4.0, .0, .0]) 

# Initialize FTOCP object
N_feas = 20
ftocp_for_mpc  = MO_FTOCP(N_feas, A, B, np.diag([100,100,0.1, 0.1]),np.diag([1, 500]))

# ====================================================================================
# Run simulation to compute feasible solution
# ====================================================================================
xcl_feasible = [x0]
ucl_feasible = []
soc_feasible = [100]
xt           = x0
time         = 0
ut = np.ones(2)

# time Loop (Perform the task until close to the origin)
while (np.dot(ut, ut) > 10**(-4) or np.dot(xt, xt) > 10**(-5)):
    xt = xcl_feasible[time] # Read measurements

    ftocp_for_mpc.solve(xt, time=time, verbose = False) # Solve FTOCP

    # Read input and apply it to the system
    ut = ftocp_for_mpc.uPred[:,0]
    
    print(f"x_t : {xt}, u_t : {ut}, soc_t: {ftocp_for_mpc.soc}")
    ucl_feasible.append(ut)
    xcl_feasible.append(ftocp_for_mpc.model(xcl_feasible[time], ut))
    soc_feasible.append(ftocp_for_mpc.soc)
    time += 1

# print(np.round(np.array(xcl_feasible).T, decimals=2))
# print(np.round(np.array(ucl_feasible).T, decimals=2))


Computing first feasible trajectory
x_t : [3. 4. 0. 0.], u_t : [-2.   -1.54], soc_t: 99.9999999735
x_t : [ 2.94  3.95 -0.5  -0.38], u_t : [-2.   -1.18], soc_t: 98.10770533556374
x_t : [ 2.75  3.82 -1.   -0.68], u_t : [-2.   -0.87], soc_t: 94.47952592602029
x_t : [ 2.44  3.62 -1.5  -0.9 ], u_t : [-2.   -0.61], soc_t: 89.23441084219526
x_t : [ 2.    3.38 -2.   -1.05], u_t : [-2.   -0.38], soc_t: 82.45847925529489
x_t : [ 1.44  3.1  -2.5  -1.14], u_t : [ 2.   -0.19], soc_t: 74.21026391825534
x_t : [ 0.87  2.81 -2.   -1.19], u_t : [ 2.   -0.04], soc_t: 67.2258021124495
x_t : [ 0.44  2.51 -1.5  -1.2 ], u_t : [2.   0.09], soc_t: 61.46152058696444
x_t : [ 0.12  2.22 -1.   -1.18], u_t : [2.   0.18], soc_t: 56.82348466940952
x_t : [-0.06  1.93 -0.5  -1.13], u_t : [2.   0.25], soc_t: 53.106948813808934
x_t : [-1.25e-01  1.65e+00 -5.81e-08 -1.07e+00], u_t : [0.71 0.3 ], soc_t: 49.89669351303703
x_t : [-0.1   1.39  0.18 -0.99], u_t : [-0.03  0.34], soc_t: 46.86752914799411
x_t : [-0.06  1.15  0.17

## Running MO-LMPC

In [44]:
N_LMPC = 5# horizon length
ftocp = MO_FTOCP(N_LMPC, A, B, Q, R, alpha=0.4) # ftocp solved by LMPC
lmpc = MO_LMPC(ftocp, CVX=True) # Initialize the LMPC (decide if you wanna use the CVX hull)
lmpc.addTrajectory(xcl_feasible, ucl_feasible) # Add feasible trajectory to the safe set
nv=len(lmpc.Q2)
totalIterations = 25 # Number of iterations to perform

# run simulation
# iteration loop
print("Starting LMPC")
for it in range(0,totalIterations):
    # Set initial condition at each iteration
    xcl = [x0] 
    xt = x0
    ftocp.soc = 100
    soccl =[ftocp.soc]
    ucl =[]
    time = 0
    
    Vp_1=lmpc.Qfun_1[0][0]
    Vp_2=[lmpc.Qfun_2[j][0][0] for j in range(nv)]
    
    Vt_1=0
    Vt_2=[0 for j in range(nv)]
    ut = np.ones(2)
    # time Loop (Perform the task until close to the origin)
    while (np.dot(xt, xt) > 10**(-3) ):

        # Read measurement
        xt = xcl[time] 
        
        # Solve FTOCP
        lmpc.solve(xt, time=time,  Vp_1=Vp_1, Vp_2=Vp_2, verbose = False) 
        # Read optimal input
        ut = lmpc.uPred[:,0]

        Vt_1+=xt.T@lmpc.Q@xt + ut.T@lmpc.R@ut 
        for j in range(nv):
            soc_consumed = lmpc.Q2[j]*np.linalg.norm(xt[2:])
            Vt_2[j]+=soc_consumed  
        # Apply optimal input  to the system
        ucl.append(ut)
        xcl.append(lmpc.ftocp.model(xcl[time], ut))
        soccl.append(lmpc.ftocp.soc)
        time += 1
        print("xt, Vt_1, Vt_2: {},{},{}".format(xt,Vt_1,Vt_2))

    # Add trajectory to update the safe set and value function
    lmpc.addTrajectory(xcl, ucl)

Trajectory added to the Safe Set. Current Iteration:  1
Trajectory cost: 
 [103.8422710725821]
SOC consumed: 
 [[72.48088286263905]]
Starting LMPC
xt, Vt_1, Vt_2: [3. 4. 0. 0.],17.299999990862197,[0.0]
xt, Vt_1, Vt_2: [ 2.94  3.94 -0.5  -0.5 ],34.12596064516589,[2.1213203415404545]
xt, Vt_1, Vt_2: [ 2.75  3.75 -1.   -0.99],47.90617744063093,[6.352769659704714]
xt, Vt_1, Vt_2: [ 2.44  3.46 -1.46 -1.31],57.651234701134946,[12.22062609927629]
xt, Vt_1, Vt_2: [ 2.06  3.11 -1.64 -1.5 ],64.7852041927289,[18.890123899701102]
xt, Vt_1, Vt_2: [ 1.64  2.73 -1.68 -1.58],70.089712063974,[25.81138036602609]
xt, Vt_1, Vt_2: [ 1.23  2.33 -1.57 -1.6 ],74.03531307561941,[32.532574268189684]
xt, Vt_1, Vt_2: [ 0.87  1.93 -1.37 -1.57],77.08469465054498,[38.796350595588834]
xt, Vt_1, Vt_2: [ 0.56  1.55 -1.11 -1.49],79.73105100379593,[44.37977757579565]
xt, Vt_1, Vt_2: [ 0.32  1.19 -0.78 -1.35],81.66422263318896,[49.06493128149706]
xt, Vt_1, Vt_2: [ 0.16  0.88 -0.49 -1.18],82.85672620653939,[52.895988508608

## Trying different objective convex combinations by varying $\alpha$

In [59]:
N_LMPC = 5# horizon length

alphas = [0.99, 0.6, 0.01]
lmpcs = []
for a in alphas:
    ftocp = MO_FTOCP(N_LMPC, A, B, Q, R, alpha=a) # ftocp solved by LMPC
    lmpc = MO_LMPC(ftocp, CVX=True) # Initialize the LMPC (decide if you wanna use the CVX hull)
    lmpc.addTrajectory(xcl_feasible, ucl_feasible) # Add feasible trajectory to the safe set
    nv=len(lmpc.Q2)
    totalIterations = 60 # Number of iterations to perform

    # run simulation
    # iteration loop
    print("Starting LMPC")
    for it in range(0,totalIterations):
        # Set initial condition at each iteration
        xcl = [x0] 
        xt = x0
        ftocp.soc = 100
        soccl =[ftocp.soc]
        ucl =[]
        time = 0
        
        Vp_1=lmpc.Qfun_1[0][0]
        Vp_2=[lmpc.Qfun_2[j][0][0] for j in range(nv)]
        
        Vt_1=0
        Vt_2=[0 for j in range(nv)]
        ut = np.ones(2)
        # time Loop (Perform the task until close to the origin)
        while (np.dot(xt, xt) > 10**(-3) ):

            # Read measurement
            xt = xcl[time] 
            
            # Solve FTOCP
            lmpc.solve(xt, time=time,  Vp_1=Vp_1, Vp_2=Vp_2, verbose = False) 
            # Read optimal input
            ut = lmpc.uPred[:,0]

            Vt_1+=xt.T@lmpc.Q@xt + ut.T@lmpc.R@ut 
            for j in range(nv):
                soc_consumed = lmpc.Q2[j]*np.linalg.norm(xt[2:])
                Vt_2[j]+=soc_consumed  
            # Apply optimal input  to the system
            ucl.append(ut)
            xcl.append(lmpc.ftocp.model(xcl[time], ut))
            soccl.append(lmpc.ftocp.soc)
            time += 1
            # print("xt, Vt_1, Vt_2: {},{},{}".format(xt,Vt_1,Vt_2))

        # Add trajectory to update the safe set and value function
        lmpc.addTrajectory(xcl, ucl)
        
    lmpcs.append(copy.copy(lmpc))
    

Trajectory added to the Safe Set. Current Iteration:  1
Trajectory cost: 
 [103.8422710725821]
SOC consumed: 
 [[72.48088286263905]]
Starting LMPC
Trajectory added to the Safe Set. Current Iteration:  2
Trajectory cost: 
 [103.8422710725821, 93.55560746377986]
SOC consumed: 
 [[72.48088286263905], [60.2372913472133]]
Trajectory added to the Safe Set. Current Iteration:  3
Trajectory cost: 
 [103.8422710725821, 93.55560746377986, 84.41367272762213]
SOC consumed: 
 [[72.48088286263905], [60.2372913472133], [60.006184738611935]]
Trajectory added to the Safe Set. Current Iteration:  4
Trajectory cost: 
 [103.8422710725821, 93.55560746377986, 84.41367272762213, 83.21913252553801]
SOC consumed: 
 [[72.48088286263905], [60.2372913472133], [60.006184738611935], [59.984050876580085]]
Trajectory added to the Safe Set. Current Iteration:  5
Trajectory cost: 
 [103.8422710725821, 93.55560746377986, 84.41367272762213, 83.21913252553801, 83.07976115107016]
SOC consumed: 
 [[72.48088286263905], [60.2

## Generating Pareto Frontier

In [71]:
N_LMPC = 5# horizon length

alphas = np.arange(0.01, 1.01, 0.03)
lmpcs_pareto = []
for a in alphas:
    ftocp = MO_FTOCP(N_LMPC, A, B, Q, R, alpha=a) # ftocp solved by LMPC
    lmpc = MO_LMPC(ftocp, CVX=True) # Initialize the LMPC (decide if you wanna use the CVX hull)
    lmpc.addTrajectory(xcl_feasible, ucl_feasible) # Add feasible trajectory to the safe set
    nv=len(lmpc.Q2)
    totalIterations = 55 # Number of iterations to perform

    # run simulation
    # iteration loop
    print("Starting LMPC")
    for it in range(0,totalIterations):
        # Set initial condition at each iteration
        xcl = [x0] 
        xt = x0
        ftocp.soc = 100
        soccl =[ftocp.soc]
        ucl =[]
        time = 0
        
        Vp_1=lmpc.Qfun_1[0][0]
        Vp_2=[lmpc.Qfun_2[j][0][0] for j in range(nv)]
        
        Vt_1=0
        Vt_2=[0 for j in range(nv)]
        ut = np.ones(2)
        # time Loop (Perform the task until close to the origin)
        while (np.dot(xt, xt) > 10**(-3) ):

            # Read measurement
            xt = xcl[time] 
            
            # Solve FTOCP
            lmpc.solve(xt, time=time,  Vp_1=Vp_1, Vp_2=Vp_2, verbose = False) 
            # Read optimal input
            ut = lmpc.uPred[:,0]

            Vt_1+=xt.T@lmpc.Q@xt + ut.T@lmpc.R@ut 
            for j in range(nv):
                soc_consumed = lmpc.Q2[j]*np.linalg.norm(xt[2:])
                Vt_2[j]+=soc_consumed  
            # Apply optimal input  to the system
            ucl.append(ut)
            xcl.append(lmpc.ftocp.model(xcl[time], ut))
            soccl.append(lmpc.ftocp.soc)
            time += 1
        
        # Add trajectory to update the safe set and value function
        lmpc.addTrajectory(xcl, ucl)
        
    lmpcs_pareto.append(copy.copy(lmpc))
    

Trajectory added to the Safe Set. Current Iteration:  1
Trajectory cost: 
 [103.8422710725821]
SOC consumed: 
 [[72.48088286263905]]
Starting LMPC
Trajectory added to the Safe Set. Current Iteration:  2
Trajectory cost: 
 [103.8422710725821, 85.75463966242778]
SOC consumed: 
 [[72.48088286263905], [65.0020475447153]]
Trajectory added to the Safe Set. Current Iteration:  3
Trajectory cost: 
 [103.8422710725821, 85.75463966242778, 84.98698077009229]
SOC consumed: 
 [[72.48088286263905], [65.0020475447153], [64.57156463020779]]
Trajectory added to the Safe Set. Current Iteration:  4
Trajectory cost: 
 [103.8422710725821, 85.75463966242778, 84.98698077009229, 84.60889507915914]
SOC consumed: 
 [[72.48088286263905], [65.0020475447153], [64.57156463020779], [64.32550262334892]]
Trajectory added to the Safe Set. Current Iteration:  5
Trajectory cost: 
 [103.8422710725821, 85.75463966242778, 84.98698077009229, 84.60889507915914, 84.30254644273316]
SOC consumed: 
 [[72.48088286263905], [65.0020

/home/shn66/.local/lib/python3.10/site-packages/cvxpy/problems/problem.py:1385: UserWarning:

Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.



Trajectory added to the Safe Set. Current Iteration:  28
Trajectory cost: 
 [103.8422710725821, 85.75463966242778, 84.98698077009229, 84.60889507915914, 84.30254644273316, 84.04250958154307, 83.82510523508769, 83.64264752628486, 83.49092566220222, 83.3624269420344, 83.2585712620989, 83.17245331083181, 83.0972393030713, 83.03132368731805, 82.97353407260223, 82.92283740676541, 82.87812520473426, 82.83860413737139, 82.80356955284665, 82.77245701482988, 82.74474330180313, 82.72000703799549, 82.69785145002609, 82.67801327757485, 82.6601710293148, 82.6440923889945, 82.62957064650183, 82.61645389190738]
SOC consumed: 
 [[72.48088286263905], [65.0020475447153], [64.57156463020779], [64.32550262334892], [64.12064088027203], [63.94747210183838], [63.800321710671476], [63.68449575917002], [63.571897879913635], [63.47287151156372], [63.38330870154021], [63.30206457340796], [63.225941585411285], [63.157772453773674], [63.09623779335057], [63.0404488608839], [62.98979125529386], [62.94388323445363],

## Plots

In [18]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create a figure
# fig = go.Figure()
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.6, 0.4],
    specs=[[{"rowspan": 1, "colspan": 1}, {"rowspan": 1, "colspan": 1}]],
    subplot_titles=("<b>X-Y Trajectories</b>", "<b>SOC% Trajectories</b>")
)

# Plot the initial trajectory
fig.add_trace(go.Scatter(
    x=[xcl_feasible[i][0] for i in range(len(xcl_feasible))],
    y=[xcl_feasible[i][1] for i in range(len(xcl_feasible))],
    mode='lines',
    name='Initial traj',
    line=dict(color='blue', width=3)
), row=1, col=1)

# Plot the final trajectory
fig.add_trace(go.Scatter(
    x=[xcl[i][0] for i in range(len(xcl))],
    y=[xcl[i][1] for i in range(len(xcl))],
    mode='lines',
    line=dict(color='green', width=3),
    name='Final traj'
), row=1, col=1)

# Plot the sets of points (SS)
for i in range(1, len(lmpc.SS)):
    fig.add_trace(go.Scatter(
        x=[lmpc.SS[i][j][0] for j in range(len(lmpc.SS[i]))],
        y=[lmpc.SS[i][j][1] for j in range(len(lmpc.SS[i]))],
        mode='markers',
        marker=dict(color='red', symbol='cross'),
        name='Safe Set',   
        showlegend=False if i>1 else True# Add label only once
    ), row=1, col=1)
    
fig.add_trace(go.Scatter(
    x=[i for i in range(len(xcl_feasible))],
    y=[soc_feasible[i] for i in range(len(xcl_feasible))],
    mode='lines',
    name='Initial traj',
    line=dict(color='blue', width=3),
    showlegend = False
), row=1, col=2)
fig.add_trace(go.Scatter(
    x=[i for i in range(len(xcl))],
    y=[soccl[i] for i in range(len(xcl))],
    mode='lines',
    line=dict(color='green', width=3),
    name='Final traj',
    showlegend = False
), row=1, col=2)

# Plot the sets of points (SS)
for i in range(1, len(lmpc.SS)):
    fig.add_trace(go.Scatter(
        x=[j for j in range(len(lmpc.socSS[i]))],
        y=[lmpc.socSS[i][j] for j in range(len(lmpc.socSS[i]))],
        mode='markers',
        marker=dict(color='red', symbol='cross'),
        name='Safe Set',   
        showlegend=False 
    ), row=1, col=2)

fig.update_xaxes(title_text="<b>X</b>", title_font=dict(size=18, family="Arial, bold"), tickfont=dict(size=16, family="Arial, bold"), row=1, col=1)
fig.update_yaxes(title_text="<b>Y</b>", title_font=dict(size=18, family="Arial, bold"), tickfont=dict(size=16, family="Arial, bold"), row=1, col=1)

fig.update_xaxes(title_text="<b>Time</b>", title_font=dict(size=18, family="Arial, bold"), tickfont=dict(size=16, family="Arial, bold"), row=1, col=2)
fig.update_yaxes(title_text="<b>SOC%</b>", title_font=dict(size=18, family="Arial, bold"), tickfont=dict(size=16, family="Arial, bold"), row=1, col=2)

# Update layouts
fig.update_layout(
    # title="<b>Trajectories in X-Y plane</b>",
    # title_font_size=24,  # Increase title font size
    width=1000,  # Width in pixels
    height=600,  # Height in pixels
    margin=dict(t=25, b=0, l=0, r=0),
    showlegend=True,
    xaxis_tickfont=dict(size=16,  family="Arial, bold"),  # Increase x-axis tick label font size
    yaxis_tickfont=dict(size=16,  family="Arial, bold"),  # Increase y-axis tick label font size
    legend_title_text='Legend',
    legend_title_font=dict(size=15),  # Increase legend title font size
    legend_font=dict(size=16)  # Increase legend items font size
)

# Show the figure
fig.show()

In [73]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=2,
    column_widths=[0.5, 0.5],
    row_heights=[0.5, 0.5],
    specs=[[{"rowspan": 2, "colspan": 1}, {"rowspan": 1, "colspan": 1}],
           [None, {"rowspan": 1, "colspan": 1}]],
    subplot_titles=("<b>SOC% vs Trajectory Cost</b>", "<b>Trajectory Cost (Iterations)</b>", "<b>SOC% Consumed (Iterations)</b>")
)

colors = ['red', 'magenta', 'goldenrod']
names =  ["alpha_0 = 0.01", "alpha_0=0.4", "alpha_0=0.99"]

pareto_costs = {0: [_l.cost_by_iter[_l.it-1][0] for _l in lmpcs_pareto], 1: [_l.cost_by_iter[_l.it-1][1][0] for _l in lmpcs_pareto]}
fig.add_trace(go.Scatter(
        x=pareto_costs[0][:-1],
        y=pareto_costs[1][:-1],
        mode='lines+markers',  # Only markers
        name="Pareto Frontier",
        line=dict(color="Green", width=2, dash="longdash")
    ), row=1, col=1)
for i,lmpc in enumerate(lmpcs):

    costs_1 = [lmpc.cost_by_iter[i][0] for i in range(0,lmpc.it)]
    costs_2 = [lmpc.cost_by_iter[i][1][0] for i in range(0,lmpc.it)]


    # Prepare the data
    iterations = list(range(1, lmpc.it))
    costs = np.array(costs_1[0:])  # Trajectory costs
    soc_consumed = np.array(costs_2[0:])  # SOC consumed

    # Create a subplot figure with 2 rows and 2 columns


    # Add the SOC vs Trajectory Cost plot to the first column spanning two rows
    fig.add_trace(go.Scatter(
        x=np.array(costs)[:50],
        y=np.array(soc_consumed)[:50],
        mode='lines+markers',  # Only markers
        name=names[i],
        line=dict(color=colors[i], width=3)
    ), row=1, col=1)

    # Add the Trajectory Cost plot to the top plot in the second column
    fig.add_trace(go.Scatter(
        x=iterations[:26],
        y=costs[:26],
        mode='lines+markers',  # Line with markers
        name='Trajectory Cost',
        line=dict(color=colors[i], width=3),
        showlegend = False
    ), row=1, col=2)

    # Add the SOC Consumed plot to the bottom plot in the second column
    fig.add_trace(go.Scatter(
        x=iterations[:26],
        y=soc_consumed[:26],
        mode='lines+markers',  # Line with markers
        name='SOC% Consumed',
        line=dict(color=colors[i], width=3),
        showlegend = False
    ), row=2, col=2)

    # Update layout
    fig.update_layout(
        # title="<b>Closed-loop Performance</b>",
        # title_font=dict(size=24, color="black", family="Arial, bold"),  # Bold title font
        width=1100,  # Adjust width to accommodate the layout
        height=600,  # Adjust height for better display
        margin=dict(t=25, b=0, l=0, r=0),
        showlegend=True
    )

    # Update x-axis and y-axis labels for clarity
    fig.update_xaxes(title_text="<b>Trajectory Cost</b>", title_font=dict(size=18, family="Arial, bold"), tickfont=dict(size=16, family="Arial, bold"), row=1, col=1)
    fig.update_yaxes(title_text="<b>SOC% Consumed</b>", title_font=dict(size=18, family="Arial, bold"), tickfont=dict(size=16, family="Arial, bold"), row=1, col=1)

    fig.update_xaxes(title_text="<b>Iterations</b>", title_font=dict(size=18, family="Arial, bold"), tickfont=dict(size=16, family="Arial, bold"), row=1, col=2)
    fig.update_yaxes(title_text="<b>Trajectory Cost</b>", title_font=dict(size=18, family="Arial, bold"), tickfont=dict(size=16, family="Arial, bold"), row=1, col=2)

    fig.update_xaxes(title_text="<b>Iterations</b>", title_font=dict(size=18, family="Arial, bold"), tickfont=dict(size=16, family="Arial, bold"), row=2, col=2)
    fig.update_yaxes(title_text="<b>SOC% Consumed</b>", title_font=dict(size=18, family="Arial, bold"), tickfont=dict(size=16, family="Arial, bold"), row=2, col=2)

    # Show the figure
fig.show()
